In [ ]:
from IPython.display import Audio

import scipy.io.wavfile as wavfile
from scipy.signal import resample
import numpy as np

In [ ]:
# import torch
# from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

In [ ]:
# device = "cuda:0" if torch.cuda.is_available() else "cpu"
# torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

In [ ]:
# device

## Load audio

In [ ]:
# Load the audio file and get the sample rate and audio data
sample_rate, audio_data = wavfile.read(f'../artifacts/e9566c13-b385-46c1-b794-d8f419628aff.wav')

# Ensure audio data is within range for int16
audio_data = np.clip(audio_data, -32768, 32767).astype(np.int16)

# audio_data is now a NumPy array containing the audio samples
# sample_rate contains the sample rate of the audio file

# You can perform operations on the audio data as needed
# For example, you can print the length of the audio in seconds:
audio_length_seconds = len(audio_data) / sample_rate
print(f"Audio Length: {audio_length_seconds} seconds")

In [ ]:
sample_rate

In [ ]:
Audio(audio_data.T, rate=sample_rate)

In [ ]:
# Define the target sample rate (16,000 Hz)
target_sample_rate = 16000

# Calculate the resampling factor
resampling_factor = target_sample_rate / sample_rate

In [ ]:
# Resample the audio data
resampled_audio_data = resample(audio_data, int(len(audio_data) * resampling_factor))

In [ ]:
Audio(resampled_audio_data.T, rate=target_sample_rate)

## Load Phi 4

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration

In [ ]:
model_id = 'openai/whisper-large-v3'

In [ ]:
processor = WhisperProcessor.from_pretrained(model_id)
model = WhisperForConditionalGeneration.from_pretrained(f'../models/{model_id}')
model.config.forced_decoder_ids = None

In [ ]:
input_values = processor(resampled_audio_data.T, 
                         return_tensors="pt",
                         sampling_rate=target_sample_rate
                        ).input_features

In [ ]:
predicted_ids = model.generate(input_values)
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)

In [ ]:
transcription[0]